In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn import preprocessing

import tensorflow as tf

In [34]:
from datetime import datetime as dt

In [198]:
from sklearn.metrics import mean_squared_error

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
pd.options.plotting.backend = "plotly"

Задача построить НС прямого распространения
На вход xi, на выход yi
Обучаем на yi, сравниваем с ti

Ошибка - Среднее квадратичное отклоенение

строим НС 10 слоев /
          20 слоев + разбивка на сигмоиде\ релу

Смотрим СКО

###

построить массив значений  множества xi=x0+i*h, где xi принадлежит отрезку [0, pi] и h=0.01
построить массив значений  множества ti=f(xi)
построить массив значений множества yi=ti+e, где е-случайно распределённая величина с нормальным распределением, матожиданием 0 и дисперсией 0.02

###

In [3]:
# функция sin

In [4]:
def sin_fun(x):
  return np.sin(2*x)

In [5]:
# формируем датасеты
def make_datasets():
  x_list = list(np.arange(0, np.pi, 0.01))
  t_list = [sin_fun(x) for x in x_list]
  y_list = [(t + (0.02 * np.random.randn() + 0)) for t in t_list] # зашумленные данные


  # dataframe чтобы рисовать можно было легко
  real_df = pd.DataFrame()
  real_df['x'] = x_list
  real_df['y'] = y_list  
  real_df['t'] = t_list
  return real_df, x_list, t_list, y_list 

In [6]:
real_df, x_list, t_list, y_list = make_datasets()

In [7]:
real_df.plot(x='x', y=['y', 't'])

In [ ]:
# TODO нужно доделать это и сделать инфраструкту 
# TODO необходимо скалировать данные
# обучаемся на y, пытаемся предсказать t
# тут два таргета

In [8]:
real_df

,x,y,t
0,0.00,-0.016711,0.000000
1,0.01,0.035308,0.019999
2,0.02,0.008449,0.039989
3,0.03,0.053989,0.059964
4,0.04,0.097224,0.079915
...,...,...,...
310,3.10,-0.050493,-0.083089
311,3.11,-0.099909,-0.063143
312,3.12,-0.018766,-0.043172
313,3.13,-0.039739,-0.023183


In [161]:
def scale_data(data_df, target_col, test_size=None):
    """
    :param data_df: dataframe with learning data
    :param target_col: column with target
    :param test_size: size to split data for test (from 0 to 1)
    :return: X_test_std, Y_test_std (scaled test data), stdsc (object of Scaler)
    """
      # objects of scaler
      # todo how to use universal scaler as object?
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_test_x = MinMaxScaler()
    scaler_test_y = MinMaxScaler()

    # splitting data in train and test datasets, also in target and features (X,Y)
    if test_size:
      train_df, test_df = train_test_split(data_df, test_size=test_size)
      train_X = train_df[list(set(train_df.columns) - set([target_col]))]
      train_Y = train_df[target_col]
      test_X = test_df[list(set(test_df.columns) - set([target_col]))]
      test_Y = test_df[target_col]

      # scaled data
      X_train_std = scaler_x.fit_transform(train_X)
      Y_train_std = scaler_y.fit_transform(np.array(train_Y).reshape(-1, 1))
      X_test_std = scaler_test_x.fit_transform(test_X)
      Y_test_std = scaler_test_y.fit_transform(np.array(test_Y).reshape(-1, 1))
      scaler_dict = {'x': scaler_x, 'y': scaler_y, 'x_test': scaler_test_x, 
                     'y_test': scaler_test_y}
      return X_train_std, Y_train_std, X_test_std, Y_test_std, scaler_dict

    else:
      train_X = data_df[list(set(data_df.columns) - set([target_col]))]
      train_Y = data_df[target_col]

      # scaled data
      X_train_std = scaler_x.fit_transform(np.array(train_X).reshape(-1, 1))
      Y_train_std = scaler_y.fit_transform(np.array(train_Y).reshape(-1, 1))
      scaler_dict = {'x': scaler_x, 'y': scaler_y}
      return X_train_std, Y_train_std, scaler_dict

In [162]:
X_train_std, Y_train_std, scaler_dict = scale_data(real_df[['x', 'y']], 'y')

In [18]:
def network_builder(layer_counts, activation_neuron):
  """
  Make model with choosen counts of layers and acitbation neuron
  layer_counts - counts of layers
  activation_neuron - neuron to activate network
  return - tf model
  """
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(layer_counts,  
                                  activation=activation_neuron))
  model.add(tf.keras.layers.Dense(1, activation='linear'))
  return model

a. 10 нейронов в скрытом слое и активационная функция сигмоида, в выходном слое будет линейная активационная функция

b. 20 нейронов в скрытом слое и активационная функция сигмоида, в выходном слое будет линейная активационная функция

c. 10 нейронов в скрытом слое и активационная функция reLu, в выходном слое будет линейная активационная функция

d. 20 нейронов в скрытом слое и активационная функция reLu, в выходном слое будет линейная активационная функция

In [30]:
model_a = network_builder(10, 'sigmoid')

In [31]:
model_b = network_builder(20, 'sigmoid')

In [32]:
model_c = network_builder(10, 'relu')

In [33]:
model_d = network_builder(20, 'relu')

In [24]:
def train_model(model_to_train, loss, metrics, X_train, y_train, epochs, batch_size):
  """
  Function to train NN model
  """
  model_to_train.compile(loss= loss, metrics=metrics, optimizer='adam')
  history = model_to_train.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
  return model_to_train, history

In [ ]:
# обучение моделей

In [ ]:
model_a_fitted = train_model(model_a, tf.keras.losses.MeanSquaredError(), 
                             tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None),
                             X_train_std, Y_train_std, 200, 16)

In [ ]:
model_b_fitted = train_model(model_b, tf.keras.losses.MeanSquaredError(), 
                             tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None),
                             X_train_std, Y_train_std, 200, 16)

In [ ]:
model_c_fitted = train_model(model_c, tf.keras.losses.MeanSquaredError(), 
                             tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None),
                             X_train_std, Y_train_std, 200, 16)

In [ ]:
model_d_fitted = train_model(model_d, tf.keras.losses.MeanSquaredError(), 
                             tf.keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None),
                             X_train_std, Y_train_std, 200, 16)

In [208]:
def print_predictions(x_list, y_list, scaler_object, models_list, names_list):
  y_predictions = {}
  y_predictions['x'] =  x_list
  y_predictions['target'] = y_list

  mse_df = pd.DataFrame()


  k = 0
  for model in models_list:
    y_predictions[names_list[k]] = np.concatenate(scaler_object['y'].inverse_transform(model.predict(x_list)), axis=0).tolist()
    mse_df[names_list[k]] = [mean_squared_error(y_list, y_predictions[names_list[k]])]
    k += 1
  result_df = pd.DataFrame.from_dict(y_predictions,orient='index').transpose()

  return result_df, mse_df

In [209]:
result_df, mse_df = print_predictions(real_df['x'].to_list(), real_df['t'].to_list(), 
                                  scaler_dict,
                                  [model_a_fitted[0], model_b_fitted[0], 
                                   model_c_fitted[0], model_d_fitted[0]], 
                                  ['model_a', 'model_b', 'model_c', 'model_d'])

10/10 [==============================] - 0s 1ms/step


In [197]:
result_df.plot(x='x', y=['target', 'model_a', 'model_b', 'model_c', 'model_d'])  # результаты не очень конеш

In [224]:
mse_df.T.plot(kind='bar', title='MSE')

In [222]:
# Resume

In [225]:
# Модели a, b дают значительно лучшие результаты, так как в них используются которая может уловить перелом функции подобно синусоиде